In [73]:
from lxml import html
import requests
import pandas as pd

In [2]:
page = requests.get('https://www.gov.uk/government/publications/register-of-licensed-sponsors-workers')
tree = html.fromstring(page.content)

In [3]:
download_path = tree.xpath('//*[@id="attachment_4271537"]/div[2]/h2/a')[0].get("href")

In [4]:
print(download_path)

https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/891847/2020-06-12_Tier_2_5_Register_of_Sponsors.pdf


In [5]:
response = requests.get(download_path)

In [6]:
with open('/tmp/metadata.pdf', 'wb') as f:
    f.write(response.content)

In [ ]:
# python3 pdf2txt.py metadata.pdf --output_type xml  --outfile raw.xml

In [ ]:
# python3 pdf2txt.py metadata.pdf --output_type html  --outfile raw.html

In [31]:
# with open('./raw-sample.html', 'r') as f:
#     page = f.read()
# doc = html.fromstring(page)

In [41]:
with open('./raw_html_all.html', 'r') as f:
    page = f.read()
doc = html.fromstring(page)

In [32]:
doc.xpath('/html/body/div[7]')[0].text

In [74]:
name = []
city = []
country = []
tier_and_rating = []
sub_tier = []

isFirst = False

for col in doc.xpath("//div/span"):
    if ("left:30px;" in col.getparent().get("style")):
        if isFirst or ("top:266px;" in col.getparent().get("style")):
            p_style = col.getparent().get("style")
            
            name.append({'style': p_style, 'name': col.text})
            isFirst = True

In [75]:
len(companies)

31324

In [76]:
companies[100]

{'name': '3H Partners Limited\n',
 'style': 'position:absolute; border: textbox 1px solid; writing-mode:lr-tb; left:30px; top:4216px; width:82px; height:8px;'}

In [77]:
df = pd.DataFrame(companies)

In [78]:
df[0:1]

,style,name
0,position:absolute; border: textbox 1px solid; ...,@ Home Accommodation Services Ltd\n


In [7]:
# for content in contents:
#     text_object = content.getObject()
#     print(text_object)

In [8]:
with open("raw_text.csv", "w") as text_file:
    text_file.write(all_text)

NameError: name 'all_text' is not defined

In [ ]:
contents = pageObj.getContents()